In [0]:
%scala

var storageAccount= "fmontes"
var container = "faviomontescontenedor"
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-05-30T15:47:40Z&st=2024-05-22T07:47:40Z&spr=https&sig=6x1Is1dTHdUKDU%2FRGyzd95d05URi4mpaO6tYsD2m%2BNg%3D"

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"

dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs = Map (conf -> sasKey))

storageAccount: String = fmontes
container: String = faviomontescontenedor
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-05-30T15:47:40Z&st=2024-05-22T07:47:40Z&spr=https&sig=6x1Is1dTHdUKDU%2FRGyzd95d05URi4mpaO6tYsD2m%2BNg%3D
conf: String = fs.azure.sas.faviomontescontenedor.fmontes.blob.core.windows.net
res3: Boolean = true

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Ingestamos archivo Producto.parquet
producto =(spark.read.parquet("/mnt/faviomontescontenedor/proyectofinal/capaplata/Producto.parquet"))

producto.createOrReplaceTempView("s_producto")

In [0]:

#Ingestamos archivo Categoria.parquet
categoria =(spark.read.parquet("/mnt/faviomontescontenedor/proyectofinal/capaplata/Categoria.parquet"))

categoria.createOrReplaceTempView("s_categoria")

In [0]:

#Ingestamos archivo SubCategoria.parquet
subcategoria =(spark.read.parquet("/mnt/faviomontescontenedor/proyectofinal/capaplata/SubCategoria.parquet"))

subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW Fact_Product
AS
SELECT Cod_Producto,P.Nombre AS NombreProducto,S.nombre AS NombreSubCategoria, C.Nombre AS NombreCategoria
FROM s_producto P
JOIN s_subcategoria S ON P.Cod_SubCategoria=S.Cod_SubCategoria
JOIN s_categoria C ON S.Cod_Categoria=C.Cod_Categoria

In [0]:
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/faviomontescontenedor/proyectofinal/capaoro/FactProducto.parquet")

In [0]:
dbutils.fs.unmount("/mnt/faviomontescontenedor")

/mnt/faviomontescontenedor has been unmounted.


True